In [1]:
!pip install geopy


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix, roc_curve, auc

# Basline Model
from geopy.distance import geodesic
from sklearn.linear_model import LogisticRegression

# Tree-Based Models
from sklearn.ensemble import RandomForestClassifier

# Gradient Boosting Models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Scaling (for Logistic Regression)
from sklearn.preprocessing import StandardScaler

In [3]:
# load the dataset and set certain arguments to read the data correctly.
data = pd.read_csv('data.csv')

# Display the first few rows of the dataset
display(data)

,name,state,latitude,longitude,category
0,Grand Canyon National Park,Arizona,36.11,-112.11,National Park
1,Yosemite National Park,California,37.87,-119.54,National Park
2,Yellowstone National Park,Wyoming,44.43,-110.59,National Park
3,Zion National Park,Utah,37.30,-113.03,National Park
4,Mount Rushmore,South Dakota,43.88,-103.46,Monument
5,Great Smoky Mountains,Tennessee,35.65,-83.51,Natural Wonder
6,Statue of Liberty,New York,40.69,-74.04,Historical Landmark
7,Arches National Park,Utah,38.73,-109.59,National Park
8,Niagara Falls,New York,43.10,-79.04,Natural Wonder
9,Golden Gate Bridge,California,37.82,-122.48,Landmark


In [4]:
# Determining the size of the DataFrame
n_rows, n_cols = data.shape
print(f"The DataFrame has {n_rows} rows and {n_cols} columns.")

The DataFrame has 10 rows and 5 columns.


In [5]:
# Display informative summary of the DataFrame
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       10 non-null     object 
 1   state      10 non-null     object 
 2   latitude   10 non-null     float64
 3   longitude  10 non-null     float64
 4   category   10 non-null     object 
dtypes: float64(2), object(3)
memory usage: 532.0+ bytes


In [6]:
# Display descriptive statistics of the DataFrame
display(data.describe())

,latitude,longitude
count,10.000000,10.000000
mean,39.558000,-102.739000
std,3.249933,17.415909
min,35.650000,-122.480000
25%,37.430000,-112.800000
50%,38.300000,-110.090000
75%,42.497500,-88.497500
max,44.430000,-74.040000


In [7]:
data.columns

Index(['name', 'state', 'latitude', 'longitude', 'category'], dtype='object')

In [8]:
locations = data[['name', 'latitude', 'longitude']]

### Baseline Route

In [9]:
# Create random route
random_route = locations.sample(frac=1, random_state=7).reset_index(drop=True)

In [21]:
# Compute total distance
def total_distance(route):
    distance = 0
    for i in range(len(route) - 1):
        start = (route.loc[i]['latitude'], route.loc[i]['longitude'])
        end = (route.loc[i + 1]['latitude'], route.loc[i + 1]['longitude'])
        distance += geodesic(start, end).km
    return distance

baseline_distance = total_distance(random_route)
print(f"Baseline Distance(km): {baseline_distance:.0f} km")

Baseline Distance(km): 13005 km


### Optimized Route